In [10]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from keys import nasa_key

Tratando a api e gerando uma lista com os dados que foram selecionados  

Criando uma nova função para consumir a api para pegarmos dados do último ano (365 dias), já que o máximo que a api consegue extrair é 7 dias por vez

In [11]:
# Chave gerada para acessar a API da Nasa
nasa_key = nasa_key

# Url API Nasa
base_url = 'https://api.nasa.gov/neo/rest/v1/feed'

# Configuração de 365 dias e intervalo de 7 dias
total_days = 365
interval_days = 7

# Lista para armazenar todos os dados de asteroides
year_asteroids = []

# Loop para dividir a solicitação em intervalos de 7 dias
for i in range(0, total_days, interval_days):
    end_date = (datetime.now() - timedelta(days=i)).strftime('%Y-%m-%d')
    start_date = (datetime.now() - timedelta(days=i + interval_days - 1)).strftime('%Y-%m-%d')
    
    params = {
        'start_date': start_date,
        'end_date': end_date,
        'api_key': nasa_key
    }
    
    # Fazer a requisição
    response = requests.get(base_url, params=params)
    data = response.json()

    # Processar os dados de cada asteroide, adicionando ao year_asteroids
    for date in data['near_earth_objects']:
        for asteroid in data['near_earth_objects'][date]:
            year_asteroids.append({
                'name': asteroid['name'],
                'close_approach_date': asteroid['close_approach_data'][0]['close_approach_date'],
                'miss_distance_km': float(asteroid['close_approach_data'][0]['miss_distance']['kilometers']),
                'relative_velocity_kmh': float(asteroid['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
                'is_potentially_hazardous': asteroid['is_potentially_hazardous_asteroid'],
                'absolute_magnitude_h': asteroid['absolute_magnitude_h'],
                'estimated_diameter_min': asteroid['estimated_diameter']['meters']['estimated_diameter_min'],
                'estimated_diameter_max': asteroid['estimated_diameter']['meters']['estimated_diameter_max'],
                'is_sentry_object': asteroid.get('is_sentry_object', False)
            })

### Explicando as variáveis  
**name**: Nome dado ao asteroide.

**close_approach_date**: Data em que se aproximou da Terra.

**relative_velocity_kmh**: Velocidade do asteroide. Velocidades maiores podem aumentar a energia do impacto, caso ele ocorra.

**estimated_diameter_min** e **estimated_diameter_max**: Intervalo de diâmetro estimado. Asteroides maiores tendem a ter um impacto mais significativo se colidirem com a Terra, o que pode correlacionar com o potencial de risco.

**estimated_diameter_mean**: Média entre diâmetro mínimo e diâmetro máximo estimado.

**miss_distance_km**: Distância mínima entre o asteroide e a Terra durante a passagem. Asteroides com menor distância de aproximação podem ser considerados mais arriscados.

**is_potentially_hazardous**: Diz se é potencialmente perigou ou não.

**absolute_magnitude_h**: Brilho absoluto do asteroide, que pode estar relacionado ao seu tamanho e ao potencial de perigo.

**is_sentry_object**: Indica se o asteroide é monitorado por ter risco de colisão.

Gerando um DataFrame a partir do 'year_asteroids'

In [12]:
df_asteroids = pd.DataFrame(year_asteroids)

Armazenando o DataFrame em um csv na pasta 'data'

In [13]:
df_asteroids.to_csv('../data/df_asteroids.csv')